<a href="https://colab.research.google.com/github/mcjackson10/BeauxSGProject/blob/main/Dog_Detector_Script_(Deep_Debug_Version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# All-in-One Dog Detector and Deterrent Script (DEEP DEBUG Version)
#
# This version is designed to troubleshoot detection issues by printing
# EVERY object the model detects, no matter how low the confidence score.
# It also prints the raw Class ID to help diagnose label mismatches.

import cv2
import numpy as np
import time
import requests
from picamera2 import Picamera2
from gpiozero import Servo, OutputDevice
from tflite_runtime.interpreter import Interpreter

# --- User-Defined Settings & Credentials ---

# Pushover Notification Credentials
PUSHOVER_USER_KEY = "ug5on3hcov7y64ioemekzk8i7p7y35"
PUSHOVER_API_TOKEN = "arxfj6m4owmuwu37x1m3tm6rqciza7"

# Model and Label File Paths
MODEL_PATH = "model.tflite"
LABEL_PATH = "labelmap.txt"

# Detection Thresholds
CONFIDENCE_THRESHOLD = 0.50  # We will adjust this after debugging
TARGET_LABEL = 'dog'

# Cooldown Period
NOTIFICATION_COOLDOWN = 30
last_notification_time = 0

# --- GPIO and Hardware Setup ---
try:
    servo = Servo(18, min_pulse_width=0.5/1000, max_pulse_width=2.5/1000)
    relay = OutputDevice(17, active_high=True, initial_value=False)
    servo.mid()
    print("GPIO components initialized successfully.")
except Exception as e:
    print(f"Error initializing GPIO components: {e}")
    print("Running in simulation mode. GPIO actions will be printed to the console.")
    servo = None
    relay = None

# --- Function Definitions ---

def send_pushover_notification(message):
    if PUSHOVER_USER_KEY == "YOUR_PUSHOVER_USER_KEY" or PUSHOVER_API_TOKEN == "YOUR_PUSHOVER_API_TOKEN":
        print("Pushover credentials not set. Skipping notification.")
        return
    try:
        payload = {
            "token": PUSHOVER_API_TOKEN,
            "user": PUSHOVER_USER_KEY,
            "message": message,
            "title": "Dog Alert!"
        }
        r = requests.post("https://api.pushover.net/1/messages.json", data=payload, timeout=5)
        r.raise_for_status()
        print("Pushover notification sent successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Failed to send Pushover notification: {e}")

def squirt_sequence():
    print("Executing squirt sequence...")
    if servo and relay:
        servo.value = -0.5
        time.sleep(0.5)
        relay.on()
        time.sleep(0.5)
        relay.off()
        time.sleep(0.5)
        servo.value = 0.5
        time.sleep(0.5)
        relay.on()
        time.sleep(0.5)
        relay.off()
        time.sleep(0.5)
        servo.mid()
    else:
        print("[SIMULATED] Squirt sequence activated.")

def load_labels(path):
    """Loads the labels file. Adds a 'background' label at index 0."""
    with open(path, 'r') as f:
        # Some models start their real labels at index 1, with 0 as a background class.
        # Prepending '???' to our list accounts for this.
        return ['???'] + [line.strip() for line in f.readlines()]

# --- Main Program ---

labels = load_labels(LABEL_PATH)
interpreter = Interpreter(MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

picam2 = Picamera2()
picam2.configure(picam2.create_preview_configuration(main={"format": 'RGB888', "size": (640, 480)}))
picam2.start()
print("Camera initialized. Starting detection loop...")

try:
    while True:
        frame = picam2.capture_array()
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        im_resized = cv2.resize(image_rgb, (width, height))
        input_data = np.expand_dims(im_resized, axis=0)

        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()

        boxes = interpreter.get_tensor(output_details[0]['index'])[0]
        classes = interpreter.get_tensor(output_details[1]['index'])[0]
        scores = interpreter.get_tensor(output_details[2]['index'])[0]

        # --- DEEP DEBUG LOGIC ---
        # Print the top 5 detections, regardless of score, to see what the model is thinking.
        print("--- Top 5 Detections (Raw) ---")
        for i in range(5):
            score = scores[i]
            class_id = int(classes[i])
            if class_id >= 0 and class_id < len(labels):
                label = labels[class_id]
                print(f"#{i+1}: ID={class_id}, Label={label}, Score={score:.2%}")
        print("------------------------------")
        # --- END DEEP DEBUG LOGIC ---

        # Loop over all detections for notification logic
        for i in range(len(scores)):
            current_score = scores[i]
            current_class_id = int(classes[i])

            if current_class_id >= 0 and current_class_id < len(labels):
                current_label = labels[current_class_id]

                if current_score > CONFIDENCE_THRESHOLD and current_label == TARGET_LABEL:
                    current_time = time.time()
                    if (current_time - last_notification_time) > NOTIFICATION_COOLDOWN:
                        print(f"--- DOG DETECTED with {current_score:.2%} confidence! ---")
                        send_pushover_notification("Dog detected in the kitchen!")
                        squirt_sequence()
                        last_notification_time = current_time

                    # Draw bounding box and label on the frame
                    ymin, xmin, ymax, xmax = boxes[i]
                    (left, right, top, bottom) = (xmin * 640, xmax * 640, ymin * 480, ymax * 480)
                    cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 2)
                    label_text = f"{current_label}: {current_score:.2%}"
                    cv2.putText(frame, label_text, (int(left), int(top) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('Object Detector', frame)

        # Press 'q' to quit
        if cv2.waitKey(1) == ord('q'):
            break

except KeyboardInterrupt:
    print("\nProgram stopped by user.")

finally:
    # Clean up resources
    cv2.destroyAllWindows()
    picam2.stop()
    if servo and relay:
        servo.detach()
        relay.close()
    print("Resources released. Program terminated gracefully.")

### **How to Use This Version**

1.  **Replace the Code:** Use `nano detector.py` to replace your script with the code above.
2.  **Run the Script:** `python detector.py`
3.  **Point the Camera at Your Dog:** Get a clear view of your dog.
4.  **Watch the Terminal VERY Closely:** Your terminal will now be flooded with debug messages every second. Look for the block that starts with `--- Top 5 Detections (Raw) ---`.

    You are looking for output like this:
    ```
    --- Top 5 Detections (Raw) ---
    #1: ID=18, Label=dog, Score=45.18%
    #2: ID=63, Label=couch, Score=15.60%
    #3: ID=1, Label=person, Score=12.11%
    #4: ID=72, Label=tv, Score=8.05%
    #5: ID=62, Label=chair, Score=5.44%
    ------------------------------
    ```

This output will tell you exactly what you need to know. Once you see the score for "dog," you can adjust the `CONFIDENCE_THRESHOLD` to be just below that val
  ```